In [ ]:
#EXAM 

##크롤링  

import urllib.request 
from bs4 import BeautifulSoup 
url = 'https://news.daum.net/society#1'


res = urllib.request.urlopen(url)   # 응답 객체
res.geturl()                         # 접속 url주소를 반환
data =res.read()                     # 객체 데이터를 읽어 data변수 저장

# utd-8 디코딩
src = data.decode('utf-8')

# html 파싱
html = BeautifulSoup(src,'html.parser')
li_tags = html.select('.tit_g > a')

crawling_data = []
i=0
for li in li_tags:
    if li.string:
        # print(f"li tag내용[{i}]: ",li.string.strip())
        crawling_data.append(li.string.strip())
    i+=1

print(crawling_data)

#top 5 단어조사 리스트저장 및 출력 
import pickle 

with open('word_count.bin','wb') as f:
    pickle.dump(crawling_data,f)  


file= open("word_count.bin", mode='rb') 
news_data= pickle.load(file)



# 텍스트 전처리
import re
def clean_text(text_str):
    # 문장부호 제거
    text_str_re = re.sub(
        '[,.?!:\'\"]',
        '',
        text_str
        )  # sub(패턴, 변경단어, 변경할 자료)
    # 특수문자, 숫자 제거
    text_str_re = re.sub(
        '[!@#$%^&*()]|[0-9]',
        '',
        text_str_re)
    # 영문 소문자 -> 영문제거
    text_str_re = text_str_re.lower()
    text_str_re = re.sub('[a-z]','',text_str_re)
    # 공백 제거
    text_str_re = ' '.join(text_str_re.split())
    
    return text_str_re

## 텍스트 전처리 함수 호출
clean_texts = [clean_text(row) for row in news_data]
# print(">> text전처리 결과 <<")
# print(clean_texts)

## word count
word_count = {}

for text in clean_texts:        # 텍스트 -> 문장
    for word in text.split():    # 문장 -> 단어
        word_count[word] = word_count.get(word,0) + 1
# print(">> 워드 카운트 <<")
# print(word_count)

# 3회 이상 출력 단어 & 2~4자 단어 지정
new_word_count = {}
for word,cnt in word_count.items():
    if cnt >= 2 and len(word) >=2 and len(word) < 4:
        print(word,'--->',word_count[word])
        new_word_count[word] = new_word_count.get(word,cnt)
print(">> 단어 전처리 끝 결과 <<")
print(new_word_count)

## top word counter
from collections import Counter     # 모듈을 추가

counter = Counter(new_word_count)
top5_word = counter.most_common(5)  # Top 5
print(">> top 5 <<")
print(top5_word) #리스트 안에 튜플 키 벨류값 구조 

# 1. 단어와 출현 빈도수 만들기
words = []       # 단어
counts = []     # 출현 빈도수

for word,count in top5_word:
    words.append(word) 
    counts.append(count)
print(words)
print(counts)

## pyplot 모듈을 import => plt로 별칭 사용
import matplotlib.pyplot as plt


## 차트에서 한글 지원
from matplotlib import font_manager,rc


rc('font', family='AppleGothic')


## 원형그래프: pie()
print('원형 그래프')
plt.pie(counts,labels=words,autopct='%.1f%%')
plt.show()

##크롤링 결과를 sqlite에 저장  

import sqlite3

try: 
    conn = sqlite3.connect('word_count_db')  
    
    cursor= conn.cursor()
    sql= 'create table if not exists word_count(no text(50),name text(50), count text(50))' 
    cursor.execute(sql)
    
    cursor.execute("insert into word_count values ('1', '내국인','2번')") 
    cursor.execute("insert into word_count values ('2', '진료','2번')") 
    cursor.execute("insert into word_count values ('3', '윤석열','2번')") 
    
    conn.commit()  

    ## 레코드 조회
    cursor.execute("select * from word_count")
    rows = cursor.fetchall()    # 조회 레코드 읽기
    # # 출력1
    for row in rows:
        print(row)
        
except Exception as e:
    print("DB연동 에러 : ",e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()